# Finding game pitchers

The purpose of this notebook is to take the `mlb_games_df.csv` file and substitute the pitcher IDs (for some site) instead of name/whatever ID is currently showing up.

In [1118]:
import pandas as pd
import numpy as np

import pybaseball as pyb

In [1119]:
games_df = pd.read_csv('../data/mlb_games_df.csv')

In [1120]:
games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,elo_diff,elo_pct_diff,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest
0,4/1/01,2001,4,1,TOR,TEX,1,loaie001,hellr001,1499.563,...,20.400,1.360396,-0.008060,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5,5
1,4/2/01,2001,4,2,SFN,SDN,1,hernl003,willw001,1540.841,...,48.041,3.117843,0.024251,0.032292,0.070273,8.784099,8.985458,14.998766,5,5
2,4/2/01,2001,4,2,SEA,OAK,1,Freddy Garcia,Tim Hudson,1519.464,...,-15.232,-1.002459,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5,5
3,4/2/01,2001,4,2,NYA,KCA,1,clemr001,suppj001,1529.511,...,36.359,2.377165,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,5,5
4,4/2/01,2001,4,2,LAN,MIL,1,parkc002,Jamey Wright,1515.925,...,25.529,1.684054,0.010660,0.014276,0.026359,4.193722,4.236467,6.181414,5,5


In [1121]:
pitchers_df = pd.read_csv('../data/pitchers_summary.csv')

In [1122]:
pitchers_df.head()

,Name,first_season,last_season,games_played,games_started,teams,num_teams,key_mlbam,key_retro,key_bbref,key_fangraphs,pitcher_cleaned
0,A.J. Burnett,2000,2015,428,423,['Marlins' 'Blue Jays' 'Yankees' 'Pirates' 'Ph...,5,150359,burna001,burnea.01,512,ajburnett
1,A.J. Cole,2015,2019,79,19,['Nationals' '- - -' 'Indians'],3,595918,colea002,coleaj01,11467,ajcole
2,A.J. Griffin,2012,2017,88,85,['Athletics' 'Rangers'],2,456167,grifa002,griffaj01,11132,ajgriffin
3,A.J. Murray,2007,2008,16,4,['Rangers'],1,451262,murra001,murraaj01,3422,ajmurray
4,Aaron Blair,2016,2017,16,16,['Braves'],1,594760,blaia001,blairaa01,14934,aaronblair


## Joining by retro key

How much of the games data has a foreign key for retrosheets?

In [1123]:
games_df['home_pitcher'].nunique(), games_df['away_pitcher'].nunique()

(1476, 1498)

In [1124]:
pd.merge(games_df, pitchers_df, left_on='home_pitcher', right_on='key_retro')['home_pitcher'].nunique()

742

In [1125]:
pd.merge(games_df, pitchers_df, left_on='away_pitcher', right_on='key_retro')['away_pitcher'].nunique()

748

So roughly half of pitchers have a useable retrosheet foreign key. Let's try names.

In [1126]:
games_df['home_pitcher_cleaned'] = games_df['home_pitcher'].str.replace(r'[\s\.\-]+', '').str.lower()
games_df['away_pitcher_cleaned'] = games_df['away_pitcher'].str.replace(r'[\s\.\-]+', '').str.lower()

In [1127]:
games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,home_pitcher_cleaned,away_pitcher_cleaned
0,4/1/01,2001,4,1,TOR,TEX,1,loaie001,hellr001,1499.563,...,-0.008060,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5,5,loaie001,hellr001
1,4/2/01,2001,4,2,SFN,SDN,1,hernl003,willw001,1540.841,...,0.024251,0.032292,0.070273,8.784099,8.985458,14.998766,5,5,hernl003,willw001
2,4/2/01,2001,4,2,SEA,OAK,1,Freddy Garcia,Tim Hudson,1519.464,...,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5,5,freddygarcia,timhudson
3,4/2/01,2001,4,2,NYA,KCA,1,clemr001,suppj001,1529.511,...,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,5,5,clemr001,suppj001
4,4/2/01,2001,4,2,LAN,MIL,1,parkc002,Jamey Wright,1515.925,...,0.010660,0.014276,0.026359,4.193722,4.236467,6.181414,5,5,parkc002,jameywright


In [1128]:
pitchers_df['pitcher_cleaned'] = pitchers_df['Name'].str.replace(r'[\s\.\-]+', '').str.lower()

In [1129]:
pitchers_df.head()

,Name,first_season,last_season,games_played,games_started,teams,num_teams,key_mlbam,key_retro,key_bbref,key_fangraphs,pitcher_cleaned
0,A.J. Burnett,2000,2015,428,423,['Marlins' 'Blue Jays' 'Yankees' 'Pirates' 'Ph...,5,150359,burna001,burnea.01,512,ajburnett
1,A.J. Cole,2015,2019,79,19,['Nationals' '- - -' 'Indians'],3,595918,colea002,coleaj01,11467,ajcole
2,A.J. Griffin,2012,2017,88,85,['Athletics' 'Rangers'],2,456167,grifa002,griffaj01,11132,ajgriffin
3,A.J. Murray,2007,2008,16,4,['Rangers'],1,451262,murra001,murraaj01,3422,ajmurray
4,Aaron Blair,2016,2017,16,16,['Braves'],1,594760,blaia001,blairaa01,14934,aaronblair


In [1130]:
pd.merge(games_df, pitchers_df, left_on='home_pitcher_cleaned', right_on='pitcher_cleaned')['home_pitcher'].nunique()

734

## Join by retrokey

It seems like almost all of them can join by retrokey or (cleaned) name. Let's first join by retrokey.

In [1131]:
def fill_missing_pitchers(left, right, left_on, right_on):
    # The only columns we need from the pitchers
    right_keys = right[['key_retro', 'key_bbref', 'pitcher_cleaned']]
    # Doing this whole "reset_index"..."set_index" preserves the indices (otherwise they get reset)
    left = left.reset_index()
    left = left.merge(right_keys, how='left', left_on=left_on, right_on=right_on)
    left = left.set_index('index')
    left = left.drop(['key_retro', 'pitcher_cleaned'], axis='columns')
    return left

In [1132]:
games_df = fill_missing_pitchers(games_df, pitchers_df, 'home_pitcher', 'key_retro')

In [1133]:
def add_key_bbref(df, prefix):
    # Rename the last column to include home/away
    cols = list(df.columns)
    cols[-1] = f'{prefix}_key_bbref'
    df.columns = cols
    return df

In [1134]:
games_df = add_key_bbref(games_df, 'home')

In [1135]:
games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,home_pitcher_cleaned,away_pitcher_cleaned,home_key_bbref
index,,,,,,,,,,,,,,,,,,,,,
0,4/1/01,2001,4,1,TOR,TEX,1,loaie001,hellr001,1499.563,...,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5,5,loaie001,hellr001,loaizes01
1,4/2/01,2001,4,2,SFN,SDN,1,hernl003,willw001,1540.841,...,0.032292,0.070273,8.784099,8.985458,14.998766,5,5,hernl003,willw001,hernali01
2,4/2/01,2001,4,2,SEA,OAK,1,Freddy Garcia,Tim Hudson,1519.464,...,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5,5,freddygarcia,timhudson,NaN
3,4/2/01,2001,4,2,NYA,KCA,1,clemr001,suppj001,1529.511,...,0.006929,0.024787,-3.703559,1.970596,5.554343,5,5,clemr001,suppj001,clemero02
4,4/2/01,2001,4,2,LAN,MIL,1,parkc002,Jamey Wright,1515.925,...,0.014276,0.026359,4.193722,4.236467,6.181414,5,5,parkc002,jameywright,parkch01


Note that there are some pitchers who have the same name but are different people.

In [1136]:
pitchers_df['key_retro'].nunique(), pitchers_df['pitcher_cleaned'].nunique()

(1671, 1668)

For now I'll just drop those pitchers to make things easier. Then I'll manually add them.

In [1137]:
pitchers_df = pitchers_df.drop_duplicates('pitcher_cleaned', keep=False)

Now we'll grab just those who didnt' join on retrokey and try joining by cleaned name.

In [1138]:
games_missing_df = games_df[games_df['home_key_bbref'].isna()]
games_missing_df = games_missing_df.drop('home_key_bbref', axis='columns')

In [1139]:
games_missing_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,home_pitcher_cleaned,away_pitcher_cleaned
index,,,,,,,,,,,,,,,,,,,,,
2,4/2/01,2001,4,2,SEA,OAK,1,Freddy Garcia,Tim Hudson,1519.464,...,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5,5,freddygarcia,timhudson
8,4/2/01,2001,4,2,MIA,PHI,0,Ryan Dempster,daalo001,1485.779,...,0.009154,0.000942,0.008884,3.540184,0.287582,2.197547,5,5,ryandempster,daalo001
9,4/2/01,2001,4,2,CLE,CHA,0,Bartolo Colon,welld001,1534.350,...,0.002157,0.010834,-0.000192,0.755044,2.978515,-0.041302,5,5,bartolocolon,welld001
24,4/4/01,2001,4,4,NYA,KCA,1,Andy Pettitte,steib002,1531.439,...,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,2,2,andypettitte,steib002
33,4/4/01,2001,4,4,MIA,PHI,0,Brad Penny,Bruce Chen,1482.669,...,0.009154,0.000942,0.008884,3.540184,0.287582,2.197547,1,1,bradpenny,brucechen


In [1140]:
games_missing_df = fill_missing_pitchers(games_missing_df, pitchers_df, 'home_pitcher_cleaned', 'pitcher_cleaned')

In [1141]:
games_missing_df = add_key_bbref(games_missing_df, 'home')

In [1142]:
games_missing_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,home_pitcher_cleaned,away_pitcher_cleaned,home_key_bbref
index,,,,,,,,,,,,,,,,,,,,,
2,4/2/01,2001,4,2,SEA,OAK,1,Freddy Garcia,Tim Hudson,1519.464,...,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5,5,freddygarcia,timhudson,garcifr03
8,4/2/01,2001,4,2,MIA,PHI,0,Ryan Dempster,daalo001,1485.779,...,0.000942,0.008884,3.540184,0.287582,2.197547,5,5,ryandempster,daalo001,dempsry01
9,4/2/01,2001,4,2,CLE,CHA,0,Bartolo Colon,welld001,1534.350,...,0.010834,-0.000192,0.755044,2.978515,-0.041302,5,5,bartolocolon,welld001,colonba01
24,4/4/01,2001,4,4,NYA,KCA,1,Andy Pettitte,steib002,1531.439,...,0.006929,0.024787,-3.703559,1.970596,5.554343,2,2,andypettitte,steib002,pettian01
33,4/4/01,2001,4,4,MIA,PHI,0,Brad Penny,Bruce Chen,1482.669,...,0.000942,0.008884,3.540184,0.287582,2.197547,1,1,bradpenny,brucechen,pennybr01


In [1143]:
games_df.update(games_missing_df)

In [1144]:
games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,home_pitcher_cleaned,away_pitcher_cleaned,home_key_bbref
index,,,,,,,,,,,,,,,,,,,,,
0,4/1/01,2001.0,4.0,1.0,TOR,TEX,1.0,loaie001,hellr001,1499.563,...,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,loaie001,hellr001,loaizes01
1,4/2/01,2001.0,4.0,2.0,SFN,SDN,1.0,hernl003,willw001,1540.841,...,0.032292,0.070273,8.784099,8.985458,14.998766,5.0,5.0,hernl003,willw001,hernali01
2,4/2/01,2001.0,4.0,2.0,SEA,OAK,1.0,Freddy Garcia,Tim Hudson,1519.464,...,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,freddygarcia,timhudson,garcifr03
3,4/2/01,2001.0,4.0,2.0,NYA,KCA,1.0,clemr001,suppj001,1529.511,...,0.006929,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,clemr001,suppj001,clemero02
4,4/2/01,2001.0,4.0,2.0,LAN,MIL,1.0,parkc002,Jamey Wright,1515.925,...,0.014276,0.026359,4.193722,4.236467,6.181414,5.0,5.0,parkc002,jameywright,parkch01


## Again for away team

In [1145]:
pitchers_df = pd.read_csv('../data/pitchers_summary.csv')
pitchers_df['pitcher_cleaned'] = pitchers_df['Name'].str.replace(r'[\s\.\-]+', '').str.lower()

games_df = fill_missing_pitchers(games_df, pitchers_df, 'away_pitcher', 'key_retro')
games_df = add_key_bbref(games_df, 'away')

pitchers_df = pitchers_df.drop_duplicates('pitcher_cleaned', keep=False)

games_missing_df = games_df[games_df['away_key_bbref'].isna()]
games_missing_df = games_missing_df.drop('away_key_bbref', axis='columns')
games_missing_df = fill_missing_pitchers(games_missing_df, pitchers_df, 'away_pitcher_cleaned', 'pitcher_cleaned')
games_missing_df = add_key_bbref(games_missing_df, 'away')

games_df.update(games_missing_df)

In [1146]:
games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,home_pitcher_cleaned,away_pitcher_cleaned,home_key_bbref,away_key_bbref
index,,,,,,,,,,,,,,,,,,,,,
0,4/1/01,2001.0,4.0,1.0,TOR,TEX,1.0,loaie001,hellr001,1499.563,...,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,loaie001,hellr001,loaizes01,helliri01
1,4/2/01,2001.0,4.0,2.0,SFN,SDN,1.0,hernl003,willw001,1540.841,...,0.070273,8.784099,8.985458,14.998766,5.0,5.0,hernl003,willw001,hernali01,williwo02
2,4/2/01,2001.0,4.0,2.0,SEA,OAK,1.0,Freddy Garcia,Tim Hudson,1519.464,...,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,freddygarcia,timhudson,garcifr03,hudsoti01
3,4/2/01,2001.0,4.0,2.0,NYA,KCA,1.0,clemr001,suppj001,1529.511,...,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,clemr001,suppj001,clemero02,suppaje01
4,4/2/01,2001.0,4.0,2.0,LAN,MIL,1.0,parkc002,Jamey Wright,1515.925,...,0.026359,4.193722,4.236467,6.181414,5.0,5.0,parkc002,jameywright,parkch01,wrighja01


## That's everyone

In [1183]:
games_df['home_pitcher'] = games_df['home_key_bbref']
games_df['away_pitcher'] = games_df['away_key_bbref']

In [1184]:
games_df['home_pitcher'].isna().sum(), games_df['away_pitcher'].isna().sum()

(0, 0)

In [1186]:
games_df = games_df.drop(['home_pitcher_cleaned', 'away_pitcher_cleaned', 'home_key_bbref', 'away_key_bbref'], axis='columns')

In [1187]:
games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,elo_diff,elo_pct_diff,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest
0,4/1/01,2001.0,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.563,...,20.400,1.360396,-0.008060,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0
1,4/2/01,2001.0,4.0,2.0,SFN,SDN,1.0,hernali01,williwo02,1540.841,...,48.041,3.117843,0.024251,0.032292,0.070273,8.784099,8.985458,14.998766,5.0,5.0
2,4/2/01,2001.0,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.464,...,-15.232,-1.002459,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0
3,4/2/01,2001.0,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.511,...,36.359,2.377165,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,5.0,5.0
4,4/2/01,2001.0,4.0,2.0,LAN,MIL,1.0,parkch01,wrighja01,1515.925,...,25.529,1.684054,0.010660,0.014276,0.026359,4.193722,4.236467,6.181414,5.0,5.0


In [1188]:
games_df.to_csv('../data/mlb_games_df.csv', index=False)